<a href="https://colab.research.google.com/github/Rodrigo-Perico/DataScience/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score


In [2]:
import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("renatosn/sao-paulo-housing-prices")

files = os.listdir(path)
print("Arquivos no dataset:", files)

csv_file = [f for f in files if f.endswith(".csv")][0]
csv_path = os.path.join(path, csv_file)

df = pd.read_csv(csv_path)

print(df.head())


100%|██████████| 211k/211k [00:00<00:00, 22.1MB/s]

Extracting files...
Arquivos no dataset: ['data.csv']


              address      district  area  bedrooms  garage  \
0          Rua Herval    Belenzinho    21         1       0   
1  Avenida São Miguel  Vila Marieta    15         1       1   
2    Rua Oscar Freire     Pinheiros    18         1       0   
3    Rua Júlio Sayago       Vila Ré    56         2       2   
4  Rua Barata Ribeiro    Bela Vista    19         1       0   

                 type  rent  total  
0     Studio e kitnet  2400   2939  
1     Studio e kitnet  1030   1345  
2         Apartamento  4000   4661  
3  Casa em condomínio  1750   1954  
4     Studio e kitnet  4000   4654  


In [3]:
ordinal_encoder = OrdinalEncoder()
adrress_encoder = ordinal_encoder.fit_transform([df["address"]])

In [4]:
adrress_encoder

array([[0., 0., 0., ..., 0., 0., 0.]])

In [5]:
from sklearn.preprocessing import OneHotEncoder


In [6]:
df = pd.get_dummies(df, columns=["area"])


print(df)

                               address                 district  bedrooms  \
0                           Rua Herval               Belenzinho         1   
1                   Avenida São Miguel             Vila Marieta         1   
2                     Rua Oscar Freire                Pinheiros         1   
3                     Rua Júlio Sayago                  Vila Ré         2   
4                   Rua Barata Ribeiro               Bela Vista         1   
...                                ...                      ...       ...   
11652          Avenida Adolfo Pinheiro              Santo Amaro         2   
11653  Rua Professor Alexandre Correia     Jardim Vitória Régia         4   
11654                 Rua Abílio Borin         Jardim Caravelas         1   
11655                Rua Forte William  Jardim Fonte do Morumbi         1   
11656              Rua Alexandre Dumas              Santo Amaro         3   

       garage                type   rent  total  area_0  area_1  area_2  ..

In [7]:
df.drop(columns=['address', 'district', 'type'], inplace=True)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
Y = df["rent"]

X = df.loc[:, df.columns != "rent"]

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [11]:
from sklearn.linear_model import LinearRegression

lin = LinearRegression()

lin.fit(x_train, y_train)

LinearRegression()

In [12]:
from sklearn.tree import DecisionTreeRegressor

In [13]:
tree = DecisionTreeRegressor()

In [14]:
tree.fit(x_train, y_train)

DecisionTreeRegressor()

In [15]:
pred = tree.predict(x_train)

In [16]:
pred

array([4125., 2200., 4500., ..., 2100., 1492., 2000.])

In [17]:
tree_mse = mean_squared_error(y_train, pred)

In [18]:
tree_mse

169.01436041998613

In [19]:
import numpy as np
tree_rmse = np.sqrt(tree_mse)


In [20]:
tree_rmse

13.00055231211298

In [21]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(tree, x_train, y_train, scoring='neg_mean_squared_error', cv=10)

In [22]:
tree_rmse_score = np.sqrt(-score)

In [23]:
print("Score:", score)
print("Mean:", score.mean())
print("Standad Deviation:", score.std())

Score: [-450277.09344363 -403050.1372549  -449474.10324755 -354296.56433824
 -368170.59558824 -578773.38970588 -500076.96844363 -437666.51286765
 -343676.96231618 -446761.53404908]
Mean: -433222.38612549624
Standad Deviation: 67677.49868627431


In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
param_grid = [

    {'splitter' :['best', 'random'],  'max_depth' : [1,5,10,20,50]},
    {'min_samples_split' : [1,2,3,9,15], 'min_samples_leaf' : [1,2,5,10,20]}

]

tree = DecisionTreeRegressor()

grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
25 fits failed out of a total of 175.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid=[{'max_depth': [1, 5, 10, 20, 50],
                          'splitter': ['best', 'random']},
                         {'min_samples_leaf': [1, 2, 5, 10, 20],
                          'min_samples_split': [1, 2, 3, 9, 15]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [26]:
final_model = grid_search.best_estimator_

final_model_predicts = final_model.predict(x_test)

final_mse = mean_squared_error(y_test, final_model_predicts)

print(np.sqrt(final_mse))

514.6320599959149


In [27]:
from sklearn.svm import SVC

In [28]:
svm = SVC()

In [29]:
svm.fit(x_train, y_train)

SVC()

In [30]:
y_pred = svm.predict(x_test)

In [33]:
print('Model accuracy score: {}'. format(accuracy_score(y_test,    y_pred)))



Model accuracy score: 0.11206403659233848
